### Bases d'apprentissage construites à partir de RISE : http://rise.life.tsinghua.edu.cn/downloads.html
- Human and Mouse Transcriptome-wide studies téléchargées depuis ce site, format .txt à renommer en .csv afin de procéder à la construction des bases de données.
- The coordinates in the database denote the interacting regions. We use hg38 (v24) and mm10 (vM9) from Genecode (https://www.gencodegenes.org). You can download the reference genome, then use bedtools getfasta / BioPython SeqIO command to retrieve the interacting regions sequence of the duplexes.  
- mouse : ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_mouse/release_M9/GRCm38.primary_assembly.genome.fa.gz
- human : ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_human/release_24/GRCh38.primary_assembly.genome.fa.gz

In [1]:
import numpy as np
import pandas as pd
import csv
from random import shuffle
from Bio import SeqIO
import RNA

Traduction des séquences d'ADN en ARN selon le strand

In [3]:
def traduire(seq,strand):
    nucleotides = ''
    if strand:
        for lettre in seq:
            if lettre=='A':
                nucleotides=nucleotides+'a'
            elif lettre=='C':
                nucleotides=nucleotides+'c'
            elif lettre=='G':
                nucleotides=nucleotides+'g'
            else:
                nucleotides=nucleotides+'u'    
    else:
        string = ''
        for i in seq:
            string = i + string
        seq = string
        for lettre in seq:
            if lettre=='A':
                nucleotides=nucleotides+'u'
            elif lettre=='C':
                nucleotides=nucleotides+'g'
            elif lettre=='G':
                nucleotides=nucleotides+'c'
            else:
                nucleotides=nucleotides+'a'
    return nucleotides

### Génération des bdd

In [28]:
# Génome humain : création d'un dictionnaire entre les chromosomes et leurs séquences dans le génome format fasta

parser = SeqIO.parse(open("genomes/GRCh38.genome.fa"),"fasta")
dict_fasta = dict([(seq.id, seq) for seq in parser])

### Création base

In [29]:
bdd = pd.read_csv("rise_human_transcriptome.csv", sep = "\t",header=None)
arrbdd = np.array(bdd)

# Ne garder que les interactions diverses (pas mRNA-mRNA)
arg_m = np.argwhere((arrbdd[:,14]!='protein_coding') | (arrbdd[:,15]!='protein_coding'))[:,0]
arg_m = np.array(arg_m)
arrbdd = arrbdd[arg_m]

#enlever les champs vides
arg_propre = np.argwhere((arrbdd[:,1] != '.') & (arrbdd[:,2] != '.') & (arrbdd[:,4] != '.') & (arrbdd[:,5] != '.') & (arrbdd[:,0] != '.') & (arrbdd[:,3] != '.') & (arrbdd[:,8] != '.') & (arrbdd[:,9] != '.'))
print(len(arg_propre))
varrbdd = arrbdd[arg_propre[:,0]]

#changement strand
for i,el in enumerate(varrbdd[:,8]):
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = True
    else:
        varrbdd[i,8] = False

for i,el in enumerate(varrbdd[:,9]):
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = True
    else:
        varrbdd[i,9] = False
arg_sansM=[]
arrbdd=[]
bdd=[]

for i,el in enumerate(varrbdd[:,1]):
        varrbdd[i,1] = int(varrbdd[i,1])
        varrbdd[i,2] = int(varrbdd[i,2])

varrbdd = varrbdd[np.argwhere(varrbdd[:,2]-varrbdd[:,1] <= 35)]

varrbdd = varrbdd[:,0]

for i,el in enumerate(varrbdd[:,4]):
        varrbdd[i,4] = int(varrbdd[i,4])
        varrbdd[i,5] = int(varrbdd[i,5])

varrbdd = varrbdd[np.argwhere(varrbdd[:,5]-varrbdd[:,4] <= 35)]

varrbdd = varrbdd[:,0]
len(varrbdd)

/media/DATA/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


33759


24229

In [30]:
bdd=varrbdd
varrbdd=[]

- Sélection de 170 nucléotides autour du site d'interaction donnée par RISE.
Seul le milieu sera utilisé pour entraînr le réseau de données mais les séquences de 200 nt serviront à remplir la base de benchmark où l'on cherchera à retrouver le site d'interaction au milieu des deux séquences
- Calcul des probabilités d'accessibilité par RNA.pl_fold sur 96 nucléotides
- Ecriture sur la base créée

In [7]:
writer = csv.writer(open("genomes/human_diversprob.csv", 'w',newline=''),delimiter="\t")

for i in range(len(bdd)):
    if i%2==0:        
        pos = bdd[i,:6]
        ide, begin, end = [bdd[i,0],bdd[i,1],bdd[i,2]]
        long1 = end-begin+1    # longueur du site d'interaction
        # 170 nt + longueur du site
        pos[0] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,8])
        w = long1 + 170
        # sélection du milieu des probabilités d'accessibilité 
        pos[2] = np.array(RNA.pfl_fold_up(pos[0][55:151],1,w,w))[1:,1][30:66]
        # deuxième séquence de l'interaction : 
        ide, begin, end = [bdd[i,3],bdd[i,4],bdd[i,5]]
        long2 = end-begin+1
        pos[1] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,9])
        w = long2 + 170
        pos[3] = np.array(RNA.pfl_fold_up(pos[1][55:151],1,w,w))[1:,1][30:66]
        pos[4] = long1
        pos[5] = long2
        writer.writerow(pos)     
    else:
        pos = bdd[i,:6]
        ide, begin, end = [bdd[i,3],bdd[i,4],bdd[i,5]]
        ide2, begin2, end2 = [bdd[i,0],bdd[i,1],bdd[i,2]]
        long1 = end-begin+1    # longueur du site d'interaction
        pos[0] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,9])
        w = long1 + 170
        if i<5:
            print(len(pos[0]),len(np.array(RNA.pfl_fold_up(pos[0][55:151],1,w,w))[1:,1]),96)
        pos[2] = np.array(RNA.pfl_fold_up(pos[0][55:151],1,w,w))[1:,1][30:66]
        # deuxième séquence de l'interaction : 
        long2 = end2-begin2+1
        pos[1] = traduire(dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[1]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[2]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[3]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[4],bdd[i,8])
        w = long2 + 170
        pos[3] = np.array(RNA.pfl_fold_up(pos[1][55:151],1,w,w))[1:,1][30:66]
        pos[4] = long1
        pos[5] = long2
        writer.writerow(pos)   
    if i%1000==0:
        print(i/len(bdd)*100,'% de la base est chargé')

203 96 96
0.0 % de la base est chargé
194 96 96
192 96 96
196 96 96
190 96 96
4.1272854843369515 % de la base est chargé
8.254570968673903 % de la base est chargé
12.381856453010855 % de la base est chargé
16.509141937347806 % de la base est chargé
20.636427421684758 % de la base est chargé
24.76371290602171 % de la base est chargé
28.890998390358664 % de la base est chargé
33.01828387469561 % de la base est chargé
37.145569359032564 % de la base est chargé
41.272854843369515 % de la base est chargé
45.40014032770647 % de la base est chargé
49.52742581204342 % de la base est chargé
53.65471129638038 % de la base est chargé
57.78199678071733 % de la base est chargé
61.90928226505428 % de la base est chargé
66.03656774939122 % de la base est chargé
70.16385323372818 % de la base est chargé
74.29113871806513 % de la base est chargé
78.41842420240208 % de la base est chargé
82.54570968673903 % de la base est chargé
86.67299517107598 % de la base est chargé
90.80028065541293 % de la base es

In [8]:
v=[]
v = pd.read_csv("genomes/human_diversprob.csv", sep = "\t",header=None)
print(len(v))
v=[]

24224


In [9]:
bdd = pd.read_csv("rise_human_transcriptome.csv", sep = "\t",header=None)
arrbdd = np.array(bdd)
bdd=[]

arg_m = np.argwhere((arrbdd[:,14]!='protein_coding') | (arrbdd[:,15]!='protein_coding'))[:,0]

arg_m = np.array(arg_m)
arrbdd = arrbdd[arg_m]

arg_propre = np.argwhere((arrbdd[:,1] != '.') & (arrbdd[:,2] != '.') & (arrbdd[:,4] != '.') & (arrbdd[:,5] != '.') & (arrbdd[:,0] != '.') & (arrbdd[:,3] != '.') & (arrbdd[:,8] != '.') & (arrbdd[:,9] != '.'))
print(len(arg_propre))
varrbdd = arrbdd[arg_propre[:,0]]

#changement strand
for i,el in enumerate(varrbdd[:,8]):
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = True
    else:
        varrbdd[i,8] = False

for i,el in enumerate(varrbdd[:,9]):
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = True
    else:
        varrbdd[i,9] = False

arrbdd=[]


for i,el in enumerate(varrbdd[:,1]):
        varrbdd[i,1] = int(varrbdd[i,1])
        varrbdd[i,2] = int(varrbdd[i,2])

varrbdd = varrbdd[np.argwhere(varrbdd[:,2]-varrbdd[:,1] <= 35)]

varrbdd = varrbdd[:,0]

for i,el in enumerate(varrbdd[:,4]):
        varrbdd[i,4] = int(varrbdd[i,4])
        varrbdd[i,5] = int(varrbdd[i,5])

varrbdd = varrbdd[np.argwhere(varrbdd[:,5]-varrbdd[:,4] <= 35)]

varrbdd = varrbdd[:,0]
len(varrbdd)

/media/DATA/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


33759


24229

In [10]:
bdd=varrbdd
varrbdd=[]

In [11]:
writer = csv.writer(open("genomes/human_divers_negatifsprob.csv", 'w',newline=''),delimiter="\t")

for i in range(len(bdd)):
    if i%2==0:        
        neg = bdd[i,:6]
        ide, begin, end = [bdd[i,0],bdd[i,1]-35,bdd[i,2]-35]
        long1 = end-begin+1    # longueur du site d'interaction
        neg[0] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,8])
        w = long1 + 170
        if i<5:
            print(len(neg[0]),len(np.array(RNA.pfl_fold_up(neg[0][55:151],1,w,w))[1:,1]),96)
        neg[2] = np.array(RNA.pfl_fold_up(neg[0][55:151],1,w,w))[1:,1][30:66]
        # deuxième séquence de l'interaction : 
        ide, begin, end = [bdd[i,3],bdd[i,4]+35,bdd[i,5]+35]
        long2 = end-begin+1
        neg[1] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,9])
        w = long2 + 170
        neg[3] = np.array(RNA.pfl_fold_up(neg[1][55:151],1,w,w))[1:,1][30:66]
        neg[4] = long1
        neg[5] = long2
        writer.writerow(neg)     
    else:
        neg = bdd[i,:6]
        ide, begin, end = [bdd[i,3],bdd[i,4]-35,bdd[i,5]-35]
        ide2, begin2, end2 = [bdd[i,0],bdd[i,1]-35,bdd[i,2]-35]
        long1 = end-begin+1    # longueur du site d'interaction
        neg[0] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,9])
        w = long1 + 170
        if i<5:
            print(len(neg[0]),len(np.array(RNA.pfl_fold_up(neg[0][55:151],1,w,w))[1:,1]),96)
        neg[2] = np.array(RNA.pfl_fold_up(neg[0][55:151],1,w,w))[1:,1][30:66]
        # deuxième séquence de l'interaction : 
        long2 = end2-begin2+1
        neg[1] = traduire(dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[1]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[2]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[3]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[4],bdd[i,8])
        w = long2 + 170
        neg[3] = np.array(RNA.pfl_fold_up(neg[1][55:151],1,w,w))[1:,1][30:66]
        neg[4] = long1
        neg[5] = long2
        writer.writerow(neg)   
    if i%1000==0:
        print(i/len(bdd)*100,'% de la base est chargé')

203 96 96
0.0 % de la base est chargé
194 96 96
192 96 96
196 96 96
190 96 96
4.1272854843369515 % de la base est chargé
8.254570968673903 % de la base est chargé
12.381856453010855 % de la base est chargé
16.509141937347806 % de la base est chargé
20.636427421684758 % de la base est chargé
24.76371290602171 % de la base est chargé
28.890998390358664 % de la base est chargé
33.01828387469561 % de la base est chargé
37.145569359032564 % de la base est chargé
41.272854843369515 % de la base est chargé
45.40014032770647 % de la base est chargé
49.52742581204342 % de la base est chargé
53.65471129638038 % de la base est chargé
57.78199678071733 % de la base est chargé
61.90928226505428 % de la base est chargé
66.03656774939122 % de la base est chargé
70.16385323372818 % de la base est chargé
74.29113871806513 % de la base est chargé
78.41842420240208 % de la base est chargé
82.54570968673903 % de la base est chargé
86.67299517107598 % de la base est chargé
90.80028065541293 % de la base es

In [17]:
v=[]
v = pd.read_csv("genomes/human_divers_negatifsprob.csv", sep = "\t",header=None)
verif = np.array(v)
print(len(v))

24229


#### Chargement base de mRNA-mRNA

In [13]:
bdd = pd.read_csv("rise_human_transcriptome.csv", sep = "\t",header=None)
arrbdd = np.array(bdd)
bdd=[]

arg_m = np.argwhere((arrbdd[:,14]=='protein_coding') & (arrbdd[:,15]=='protein_coding'))[:,0]

arg_m = np.array(arg_m)
arrbdd = arrbdd[arg_m]

arg_propre = np.argwhere((arrbdd[:,1] != '.') & (arrbdd[:,2] != '.') & (arrbdd[:,4] != '.') & (arrbdd[:,5] != '.') & (arrbdd[:,0] != '.') & (arrbdd[:,3] != '.') & (arrbdd[:,8] != '.') & (arrbdd[:,9] != '.'))
print(len(arg_propre))
varrbdd = arrbdd[arg_propre[:,0]]

#changement strand
for i,el in enumerate(varrbdd[:,8]):
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = True
    else:
        varrbdd[i,8] = False

for i,el in enumerate(varrbdd[:,9]):
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = True
    else:
        varrbdd[i,9] = False

arrbdd=[]


for i,el in enumerate(varrbdd[:,1]):
        varrbdd[i,1] = int(varrbdd[i,1])
        varrbdd[i,2] = int(varrbdd[i,2])

varrbdd = varrbdd[np.argwhere(varrbdd[:,2]-varrbdd[:,1] <= 35)]

varrbdd = varrbdd[:,0]

for i,el in enumerate(varrbdd[:,4]):
        varrbdd[i,4] = int(varrbdd[i,4])
        varrbdd[i,5] = int(varrbdd[i,5])

varrbdd = varrbdd[np.argwhere(varrbdd[:,5]-varrbdd[:,4] <= 35)]

varrbdd = varrbdd[:,0]
len(varrbdd)

/media/DATA/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


39643


27078

In [14]:
bdd=varrbdd
varrbdd=[]

In [15]:
writer = csv.writer(open("genomes/human_mRNA-mRNAprob.csv", 'w',newline=''),delimiter="\t")

for i in range(len(bdd)):
    if i%2==0:        
        pos = bdd[i,:6]
        ide, begin, end = [bdd[i,0],bdd[i,1],bdd[i,2]]
        long1 = end-begin+1    # longueur du site d'interaction
        pos[0] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,8])
        w = long1 + 170
        if i<5:
            print(len(pos[0]),len(np.array(RNA.pfl_fold_up(pos[0][55:151],1,w,w))[1:,1]),96)
        pos[2] = np.array(RNA.pfl_fold_up(pos[0][55:151],1,w,w))[1:,1][30:66]
        # deuxième séquence de l'interaction : 
        ide, begin, end = [bdd[i,3],bdd[i,4],bdd[i,5]]
        long2 = end-begin+1
        pos[1] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,9])
        w = long2 + 170
        pos[3] = np.array(RNA.pfl_fold_up(pos[1][55:151],1,w,w))[1:,1][30:66]
        pos[4] = long1
        pos[5] = long2
        writer.writerow(pos)     
    else:
        pos = bdd[i,:6]
        ide, begin, end = [bdd[i,3],bdd[i,4],bdd[i,5]]
        ide2, begin2, end2 = [bdd[i,0],bdd[i,1],bdd[i,2]]
        long1 = end-begin+1    # longueur du site d'interaction
        pos[0] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,9])
        w = long1 + 170
        if i<5:
            print(len(pos[0]),len(np.array(RNA.pfl_fold_up(pos[0][55:151],1,w,w))[1:,1]),96)
        pos[2] = np.array(RNA.pfl_fold_up(pos[0][55:151],1,w,w))[1:,1][30:66]
        # deuxième séquence de l'interaction : 
        long2 = end2-begin2+1
        pos[1] = traduire(dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[1]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[2]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[3]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[4],bdd[i,8])
        w = long2 + 170
        pos[3] = np.array(RNA.pfl_fold_up(pos[1][55:151],1,w,w))[1:,1][30:66]
        pos[4] = long1
        pos[5] = long2
        writer.writerow(pos)   
    if i%1000==0:
        print(i/len(bdd)*100,'% de la base est chargé')

205 96 96
0.0 % de la base est chargé
191 96 96
205 96 96
199 96 96
191 96 96
3.6930349361104953 % de la base est chargé
7.3860698722209905 % de la base est chargé
11.079104808331488 % de la base est chargé
14.772139744441981 % de la base est chargé
18.46517468055248 % de la base est chargé
22.158209616662976 % de la base est chargé
25.851244552773466 % de la base est chargé
29.544279488883962 % de la base est chargé
33.237314424994466 % de la base est chargé
36.93034936110496 % de la base est chargé
40.62338429721545 % de la base est chargé
44.31641923332595 % de la base est chargé
48.009454169436445 % de la base est chargé
51.70248910554693 % de la base est chargé
55.39552404165743 % de la base est chargé
59.088558977767924 % de la base est chargé
62.78159391387843 % de la base est chargé
66.47462884998893 % de la base est chargé
70.16766378609942 % de la base est chargé
73.86069872220992 % de la base est chargé
77.55373365832041 % de la base est chargé
81.2467685944309 % de la base 

In [35]:
v=[]
v = pd.read_csv("genomes/human_mRNA-mRNAprob.csv", sep = "\t",header=None)
verif = np.array(v)
print(len(v))

27074


In [36]:
bdd = pd.read_csv("rise_human_transcriptome.csv", sep = "\t",header=None)
arrbdd = np.array(bdd)
bdd=[]

arg_m = np.argwhere((arrbdd[:,14]=='protein_coding') & (arrbdd[:,15]=='protein_coding'))[:,0]

arg_m = np.array(arg_m)
arrbdd = arrbdd[arg_m]

arg_propre = np.argwhere((arrbdd[:,1] != '.') & (arrbdd[:,2] != '.') & (arrbdd[:,4] != '.') & (arrbdd[:,5] != '.') & (arrbdd[:,0] != '.') & (arrbdd[:,3] != '.') & (arrbdd[:,8] != '.') & (arrbdd[:,9] != '.'))
print(len(arg_propre))
varrbdd = arrbdd[arg_propre[:,0]]

#changement strand
for i,el in enumerate(varrbdd[:,8]):
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = True
    else:
        varrbdd[i,8] = False

for i,el in enumerate(varrbdd[:,9]):
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = True
    else:
        varrbdd[i,9] = False

arrbdd=[]


for i,el in enumerate(varrbdd[:,1]):
        varrbdd[i,1] = int(varrbdd[i,1])
        varrbdd[i,2] = int(varrbdd[i,2])

varrbdd = varrbdd[np.argwhere(varrbdd[:,2]-varrbdd[:,1] <= 35)]

varrbdd = varrbdd[:,0]

for i,el in enumerate(varrbdd[:,4]):
        varrbdd[i,4] = int(varrbdd[i,4])
        varrbdd[i,5] = int(varrbdd[i,5])

varrbdd = varrbdd[np.argwhere(varrbdd[:,5]-varrbdd[:,4] <= 35)]

varrbdd = varrbdd[:,0]
len(varrbdd)

/media/DATA/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


39643


27078

In [37]:
bdd = varrbdd
varrbdd = []

In [38]:
writer = csv.writer(open("genomes/human_mRNA-mRNA_negatifsprob.csv", 'w',newline=''),delimiter="\t")

for i in range(len(bdd)):
    if i%2==0:        
        neg = bdd[i,:6]
        ide, begin, end = [bdd[i,0],bdd[i,1]-35,bdd[i,2]-35]
        long1 = end-begin+1    # longueur du site d'interaction
        neg[0] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,8])
        w = long1 + 170
        if i<5:
            print(len(neg[0]),len(np.array(RNA.pfl_fold_up(neg[0][55:151],1,w,w))[1:,1]),96)
        neg[2] = np.array(RNA.pfl_fold_up(neg[0][55:151],1,w,w))[1:,1][30:66]
        # deuxième séquence de l'interaction : 
        ide, begin, end = [bdd[i,3],bdd[i,4]+35,bdd[i,5]+35]
        long2 = end-begin+1
        neg[1] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,9])
        w = long2 + 170
        neg[3] = np.array(RNA.pfl_fold_up(neg[1][55:151],1,w,w))[1:,1][30:66]
        neg[4] = long1
        neg[5] = long2
        writer.writerow(neg)     
    else:
        neg = bdd[i,:6]
        ide, begin, end = [bdd[i,3],bdd[i,4]-35,bdd[i,5]-35]
        ide2, begin2, end2 = [bdd[i,0],bdd[i,1]-35,bdd[i,2]-35]
        long1 = end-begin+1    # longueur du site d'interaction
        neg[0] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,9])
        w = long1 + 170
        if i<5:
            print(len(neg[0]),len(np.array(RNA.pfl_fold_up(neg[0][55:151],1,w,w))[1:,1]),96)
        neg[2] = np.array(RNA.pfl_fold_up(neg[0][55:151],1,w,w))[1:,1][30:66]
        # deuxième séquence de l'interaction : 
        long2 = end2-begin2+1
        neg[1] = traduire(dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[1]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[2]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[3]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[4],bdd[i,8])
        w = long2 + 170
        neg[3] = np.array(RNA.pfl_fold_up(neg[1][55:151],1,w,w))[1:,1][30:66]
        neg[4] = long1
        neg[5] = long2
        writer.writerow(neg)   
    if i%1000==0:
        print(i/len(bdd)*100,'% de la base est chargé')

205 96 96
0.0 % de la base est chargé
191 96 96
205 96 96
199 96 96
191 96 96
3.6930349361104953 % de la base est chargé
7.3860698722209905 % de la base est chargé
11.079104808331488 % de la base est chargé
14.772139744441981 % de la base est chargé
18.46517468055248 % de la base est chargé
22.158209616662976 % de la base est chargé
25.851244552773466 % de la base est chargé
29.544279488883962 % de la base est chargé
33.237314424994466 % de la base est chargé
36.93034936110496 % de la base est chargé
40.62338429721545 % de la base est chargé
44.31641923332595 % de la base est chargé
48.009454169436445 % de la base est chargé
51.70248910554693 % de la base est chargé
55.39552404165743 % de la base est chargé
59.088558977767924 % de la base est chargé
62.78159391387843 % de la base est chargé
66.47462884998893 % de la base est chargé
70.16766378609942 % de la base est chargé
73.86069872220992 % de la base est chargé
77.55373365832041 % de la base est chargé
81.2467685944309 % de la base 

In [39]:
v=[]
v = pd.read_csv("genomes/human_mRNA-mRNA_negatifsprob.csv", sep = "\t",header=None)
verif = np.array(v)
print(len(v))

27077


#### Génome de la souris

In [40]:
# génome de la souris
parser = SeqIO.parse(open("genomes/GRCm38.fa"),"fasta")
dict_fasta = dict([(seq.id, seq) for seq in parser])

In [41]:
bdd = pd.read_csv("rise_mouse.csv", sep = "\t",header=None)
arrbdd = np.array(bdd)

arg_propre = np.argwhere((arrbdd[:,1] != '.') & (arrbdd[:,2] != '.') & (arrbdd[:,4] != '.') & (arrbdd[:,5] != '.') & (arrbdd[:,0] != '.') & (arrbdd[:,3] != '.') & (arrbdd[:,8] != '.') & (arrbdd[:,9] != '.'))
print(len(arg_propre))
varrbdd = arrbdd[arg_propre[:,0]]


#changement strand
for i,el in enumerate(varrbdd[:,8]):
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = True
    else:
        varrbdd[i,8] = False

for i,el in enumerate(varrbdd[:,9]):
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = True
    else:
        varrbdd[i,9] = False
arg_sansM=[]
arrbdd=[]
bdd=[]

for i,el in enumerate(varrbdd[:,1]):
        varrbdd[i,1] = int(varrbdd[i,1])
        varrbdd[i,2] = int(varrbdd[i,2])

varrbdd = varrbdd[np.argwhere(varrbdd[:,2]-varrbdd[:,1] <= 35)]

varrbdd = varrbdd[:,0]

for i,el in enumerate(varrbdd[:,4]):
        varrbdd[i,4] = int(varrbdd[i,4])
        varrbdd[i,5] = int(varrbdd[i,5])

varrbdd = varrbdd[np.argwhere(varrbdd[:,5]-varrbdd[:,4] <= 35)]

varrbdd = varrbdd[:,0]
len(varrbdd)

173412


37993

In [42]:
bdd=varrbdd
varrbdd=[]

In [43]:
writer = csv.writer(open("genomes/mouse_divers_posprob.csv", 'w',newline=''),delimiter="\t")

for i in range(len(bdd)):
    if i%2==0:        
        pos = bdd[i,:6]
        ide, begin, end = [bdd[i,0],bdd[i,1],bdd[i,2]]
        long1 = end-begin+1    # longueur du site d'interaction
        pos[0] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,8])
        w = long1 + 170
        if i<5:
            print(len(pos[0]),len(np.array(RNA.pfl_fold_up(pos[0][55:151],1,w,w))[1:,1]),96)
        pos[2] = np.array(RNA.pfl_fold_up(pos[0][55:151],1,w,w))[1:,1][30:66]
        # deuxième séquence de l'interaction : 
        ide, begin, end = [bdd[i,3],bdd[i,4],bdd[i,5]]
        long2 = end-begin+1
        pos[1] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,9])
        w = long2 + 170
        pos[3] = np.array(RNA.pfl_fold_up(pos[1][55:151],1,w,w))[1:,1][30:66]
        pos[4] = long1
        pos[5] = long2
        writer.writerow(pos)     
    else:
        pos = bdd[i,:6]
        ide, begin, end = [bdd[i,3],bdd[i,4],bdd[i,5]]
        ide2, begin2, end2 = [bdd[i,0],bdd[i,1],bdd[i,2]]
        long1 = end-begin+1    # longueur du site d'interaction
        pos[0] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,9])
        w = long1 + 170
        if i<5:
            print(len(pos[0]),len(np.array(RNA.pfl_fold_up(pos[0][55:151],1,w,w))[1:,1]),96)
        pos[2] = np.array(RNA.pfl_fold_up(pos[0][55:151],1,w,w))[1:,1][30:66]
        # deuxième séquence de l'interaction : 
        long2 = end2-begin2+1
        pos[1] = traduire(dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[1]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[2]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[3]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[4],bdd[i,8])
        w = long2 + 170
        pos[3] = np.array(RNA.pfl_fold_up(pos[1][55:151],1,w,w))[1:,1][30:66]
        pos[4] = long1
        pos[5] = long2
        writer.writerow(pos)   
    if i%1000==0:
        print(i/len(bdd)*100,'% de la base est chargé')

194 96 96
0.0 % de la base est chargé
200 96 96
189 96 96
189 96 96
191 96 96
2.6320638012265416 % de la base est chargé
5.264127602453083 % de la base est chargé
7.896191403679626 % de la base est chargé
10.528255204906166 % de la base est chargé
13.16031900613271 % de la base est chargé
15.792382807359251 % de la base est chargé
18.424446608585793 % de la base est chargé
21.056510409812333 % de la base est chargé
23.688574211038876 % de la base est chargé
26.32063801226542 % de la base est chargé
28.95270181349196 % de la base est chargé
31.584765614718503 % de la base est chargé
34.21682941594504 % de la base est chargé
36.848893217171586 % de la base est chargé
39.48095701839813 % de la base est chargé
42.113020819624666 % de la base est chargé
44.74508462085121 % de la base est chargé
47.37714842207775 % de la base est chargé
50.00921222330429 % de la base est chargé
52.64127602453084 % de la base est chargé
55.273339825757375 % de la base est chargé
57.90540362698392 % de la base

In [44]:
v=[]
v = pd.read_csv("genomes/mouse_divers_posprob.csv", sep = "\t",header=None)
verif = np.array(v)
print(len(v))

37989


In [45]:
bdd = pd.read_csv("rise_mouse.csv", sep = "\t",header=None)
arrbdd = np.array(bdd)

arg_propre = np.argwhere((arrbdd[:,1] != '.') & (arrbdd[:,2] != '.') & (arrbdd[:,4] != '.') & (arrbdd[:,5] != '.') & (arrbdd[:,0] != '.') & (arrbdd[:,3] != '.') & (arrbdd[:,8] != '.') & (arrbdd[:,9] != '.'))
print(len(arg_propre))
varrbdd = arrbdd[arg_propre[:,0]]


#changement strand
for i,el in enumerate(varrbdd[:,8]):
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = True
    else:
        varrbdd[i,8] = False

for i,el in enumerate(varrbdd[:,9]):
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = True
    else:
        varrbdd[i,9] = False
arg_sansM=[]
arrbdd=[]
bdd=[]

for i,el in enumerate(varrbdd[:,1]):
        varrbdd[i,1] = int(varrbdd[i,1])
        varrbdd[i,2] = int(varrbdd[i,2])

varrbdd = varrbdd[np.argwhere(varrbdd[:,2]-varrbdd[:,1] <= 35)]

varrbdd = varrbdd[:,0]

for i,el in enumerate(varrbdd[:,4]):
        varrbdd[i,4] = int(varrbdd[i,4])
        varrbdd[i,5] = int(varrbdd[i,5])

varrbdd = varrbdd[np.argwhere(varrbdd[:,5]-varrbdd[:,4] <= 35)]

varrbdd = varrbdd[:,0]
len(varrbdd)

173412


37993

In [46]:
bdd=varrbdd
varrbdd=[]

In [47]:
writer = csv.writer(open("genomes/mouse_divers_negprob.csv", 'w',newline=''),delimiter="\t")

for i in range(len(bdd)):
    if i%2==0:        
        neg = bdd[i,:6]
        ide, begin, end = [bdd[i,0],bdd[i,1]-35,bdd[i,2]-35]
        long1 = end-begin+1    # longueur du site d'interaction
        neg[0] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,8])
        w = long1 + 170
        if i<5:
            print(len(neg[0]),len(np.array(RNA.pfl_fold_up(neg[0][55:151],1,w,w))[1:,1]),96)
        neg[2] = np.array(RNA.pfl_fold_up(neg[0][55:151],1,w,w))[1:,1][30:66]
        # deuxième séquence de l'interaction : 
        ide, begin, end = [bdd[i,3],bdd[i,4]+35,bdd[i,5]+35]
        long2 = end-begin+1
        neg[1] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,9])
        w = long2 + 170
        neg[3] = np.array(RNA.pfl_fold_up(neg[1][55:151],1,w,w))[1:,1][30:66]
        neg[4] = long1
        neg[5] = long2
        writer.writerow(neg)     
    else:
        neg = bdd[i,:6]
        ide, begin, end = [bdd[i,3],bdd[i,4]-35,bdd[i,5]-35]
        ide2, begin2, end2 = [bdd[i,0],bdd[i,1]-35,bdd[i,2]-35]
        long1 = end-begin+1    # longueur du site d'interaction
        neg[0] = traduire(dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[1]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[2]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[3]+dict_fasta[ide][begin-85:end+1+85].format("fasta").split('\n')[4],bdd[i,9])
        w = long1 + 170
        if i<5:
            print(len(neg[0]),len(np.array(RNA.pfl_fold_up(neg[0][55:151],1,w,w))[1:,1]),96)
        neg[2] = np.array(RNA.pfl_fold_up(neg[0][55:151],1,w,w))[1:,1][30:66]
        # deuxième séquence de l'interaction : 
        long2 = end2-begin2+1
        neg[1] = traduire(dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[1]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[2]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[3]+dict_fasta[ide2][begin2-85:end2+1+85].format("fasta").split('\n')[4],bdd[i,8])
        w = long2 + 170
        neg[3] = np.array(RNA.pfl_fold_up(neg[1][55:151],1,w,w))[1:,1][30:66]
        neg[4] = long1
        neg[5] = long2
        writer.writerow(neg)   
    if i%1000==0:
        print(i/len(bdd)*100,'% de la base est chargé')

194 96 96
0.0 % de la base est chargé
200 96 96
189 96 96
189 96 96
191 96 96
2.6320638012265416 % de la base est chargé
5.264127602453083 % de la base est chargé
7.896191403679626 % de la base est chargé
10.528255204906166 % de la base est chargé
13.16031900613271 % de la base est chargé
15.792382807359251 % de la base est chargé
18.424446608585793 % de la base est chargé
21.056510409812333 % de la base est chargé
23.688574211038876 % de la base est chargé
26.32063801226542 % de la base est chargé
28.95270181349196 % de la base est chargé
31.584765614718503 % de la base est chargé
34.21682941594504 % de la base est chargé
36.848893217171586 % de la base est chargé
39.48095701839813 % de la base est chargé
42.113020819624666 % de la base est chargé
44.74508462085121 % de la base est chargé
47.37714842207775 % de la base est chargé
50.00921222330429 % de la base est chargé
52.64127602453084 % de la base est chargé
55.273339825757375 % de la base est chargé
57.90540362698392 % de la base

In [48]:
v=[]
v = pd.read_csv("genomes/mouse_divers_negprob.csv", sep = "\t",header=None)
verif = np.array(v)
print(len(v))

37990
